In [1]:
%%js
window.R3UP = 
{
 author:'r3ki3g',
activation_function:'ReLu'
 
};

fetch('http://localhost:8888/files/PROJECTS/_r3_jupytor_js_ext/jq_and_code.js?download=1').then(response => response.text()).then(script => eval(script)).catch(error => alert(error));




<IPython.core.display.Javascript object>

## Self coding/implementation of a 3 layer neural network (activation={{activation_function}})
The original version is at backup (10)


In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def sigmoid(t):return 1/(1+np.exp(-t))
def ReLu(t):return np.max((t,np.zeros(t.shape)),axis=0)

### Try a simple classification to verify it's correct

<font color="red">I am so happy this code implementation worked as I expected even without any debugging.\
    WOW!
</font>


In [4]:
def randomToAmplitude(n,A=10):
    return A * (2*n-1)

In [5]:
#define the node counts
N = 1 #number of input elements
H = 100 #number of hidden elements
M = 1 #number of output elements

#data-set stuff
# m = 1000 #number of examples
# x_train = np.random.rand(m,N)
# y_train = (x_train>0.5).astype(np.int16)

m = 1000

pos_x = 0.25 + 0.5 * np.random.rand(m//2,1)
neg_x1 = 0 + 0.25 * np.random.rand(m//4,1)
neg_x2 = 0.75 + 0.25 * np.random.rand(m//4,1)
neg_x = np.concatenate((neg_x1,neg_x2))

x_train = np.concatenate((pos_x,neg_x))
y_train = np.logical_and(x_train>0.25,x_train<0.75).astype(np.int16)



x_y = np.hstack((x_train.reshape(-1,1),y_train.reshape(-1,1)))
print(x_y)






#our convension of x,y is different
x = x_train.T
y = y_train.T


def fit():
    #initialize the parameters as arrays with proper dimensions
    W = randomToAmplitude(np.random.rand(H,N),10)
    b = randomToAmplitude(np.random.rand(H),10)
    V = randomToAmplitude(np.random.rand(M,H),10)
    c = randomToAmplitude(np.random.rand(M),10)

    cost_history = []

    n_iters = 50_000
    for _ in range(n_iters):

        #forward pass calculations
        h = sigmoid(W @ x + b.reshape(-1,1))
        y_hat = sigmoid(V @ (h) + c.reshape(-1,1))

        #y_hat_alpha - y_alpha
        diff = y_hat - y
        #dSigmoidFactor_y_hat = y_hat * (1 - y_hat)
        dReLu_factor = (y_hat>0).astype(np.float64)

        intermediate_0 = diff * dReLu_factor

        #fine tune c
        dE_dcALLAlpha = np.sum(intermediate_0, axis = 1)/m #dim = (M,)
        ##completely found :E_dcALLAlpha


        dE_dVALLAlphaALLBeta = np.zeros((M,H))
        for beta in range(H):
            dE_dVALLAlphaThisBeta = np.sum(intermediate_0 * h[beta,:], axis=1) /m
            dE_dVALLAlphaALLBeta[:,beta] = dE_dVALLAlphaThisBeta
        ##completely found :dE_dVALLAlphaALLBeta


        dE_dbALLAlpha = np.zeros(H)
        dE_dWALLAlpha_ALLBeta = np.zeros((H,N))

        for alpha in range(H):

    #         dSigmoidFactor_h_alpha = h[alpha,:] #temp
    #         dSigmoidFactor_h_alpha *= 1- dSigmoidFactor_h_alpha #done

            dReLuFactor_h_alpha = (h[alpha,:]>0).astype(np.float64)

            intermediate_1 = intermediate_0 * V[:,alpha].reshape(-1,1) * dReLuFactor_h_alpha
            dE_dbThisAlpha= np.sum(intermediate_1)/m
            dE_dbALLAlpha[alpha] = dE_dbThisAlpha

            for beta in range(N):
                dE_dWThisAlphaThisBeta= np.sum(intermediate_1 * x[beta,:])/m
                dE_dWALLAlpha_ALLBeta[alpha,beta] = dE_dWThisAlphaThisBeta
        ##completely found :dE_dbALLAlpha
        ##completely found :dE_dWALLAlpha_ALLBeta


        #All derivatives are now found

        #calc the cost now
        cost = np.sum(y_hat - y)/(2*m)
        cost_history.append(cost)

        ## Update parameters :: Gradient decesnt
        lr = 1e-2

        c -= lr * dE_dcALLAlpha
        V -= lr * dE_dVALLAlphaALLBeta
        b -= lr * dE_dbALLAlpha
        W -= lr * dE_dWALLAlpha_ALLBeta




    print("Calculated weights)")
    print("c:",c)
    print("V:",V)
    print("b:",b)
    print("W:",W)
    plt.plot(cost_history)
    plt.show()
    #print("Final cost is:",cost_history[-1])
    return (c,V,b,W)#,cost_history


[[0.47211701 1.        ]
 [0.61841031 1.        ]
 [0.65843589 1.        ]
 ...
 [0.81366577 0.        ]
 [0.9131291  0.        ]
 [0.78527027 0.        ]]


In [6]:
def predict(x_test,c,V,b,W):
    x = x_test.T
    h = ReLu(W @ x + b.reshape(-1,1))
    y_hat = ReLu(V @ (h) + c.reshape(-1,1))
    y_predicted = y_hat.T>0.5
    return y_predicted,x,h,y_hat


In [7]:
if not 'need':
    #calc accuracy
    y_pred = predict(x_train)[0]
    accuracy = np.sum(y_pred == y_train)/np.prod(y_pred.shape)
    print("Accuracy:", accuracy)

In [8]:
def fit_and_eval4accuracy():
    (c,V,b,W) = fit()
    y_pred = predict(x_train,c,V,b,W)[0]
    accuracy = np.sum(y_pred == y_train)/np.prod(y_pred.shape)
    
#     fig,ax = plt.subplots(1)
#     ax.plot(cost_history)
#     ax.title(f'final accuracy : {accuracy}')
#     plt.show()
    
    return accuracy
    

In [9]:
#parallely do multiple times
from joblib import Parallel, delayed
results = []
for _ in range(1):
    results.extend(Parallel(n_jobs=20)((

        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)(),
        delayed(fit_and_eval4accuracy)()
        
    )))
    
print("max accuracy:",max(results))

C:\Users\R3KI3G\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


max accuracy: 0.73


In [10]:
if not 'need':
    xx = np.array([i/100 for i in range(0,100,3)]).reshape(-1,1)
    yy = predict(xx)

    visualizer = np.concatenate((xx,yy[0]),axis=1)
    print(visualizer)

In [11]:
yy[1].T

NameError: name 'yy' is not defined

In [ ]:
yy[0]


In [ ]:
yy[2].T